In [28]:
import pandas as pd 

# ティッカーリスト
tickers = [7203, 9984]
tickers = pd.read_csv(fr'C:\Users\100ca\Documents\PyCode\IR\data\tickers.csv')['コード'].to_list()

In [29]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import csv

# 企業ごとの給与情報を取得する関数
def fetch_salary_data(ticker):
    base_url = f'https://irbank.net/{ticker}/salary'
    response = requests.get(base_url)
    return response.text if response.ok else None

# 給与情報を解析する関数
def parse_salary_data(html_content, ticker):
    if not html_content:
        return {'ticker': ticker, 'company_name': '情報なし', 'average_salary': None, 'average_age': '情報なし'}
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 企業名を取得
    company_title_tag = soup.select_one('header nav ol li a[title]')
    company_name = company_title_tag.get('title').split(' ')[1] if company_title_tag and ' ' in company_title_tag.get('title', '') else '情報なし'
    
    # 平均給与を取得し、'万'を取り除き整数に変換
    average_salary_tag = soup.select_one('p.pad strong')
    if average_salary_tag and '万' in average_salary_tag.get_text(strip=True):
        average_salary = int(average_salary_tag.get_text(strip=True).replace('万', ''))
    else:
        average_salary = None
    
    details_text = soup.select_one('p.pad').get_text(strip=True) if soup.select_one('p.pad') else ''
    
    # 平均年齢を取得
    average_age = ''
    if '平均年齢' in details_text:
        average_age = details_text.split('平均年齢')[1].split('歳')[0].strip() if '歳' in details_text else '情報なし'
    
    return {'ticker': ticker, 'company_name': company_name, 'average_salary': average_salary, 'average_age': average_age}

# 企業の給与情報を取得し、解析する関数
def fetch_and_parse_salary_data(ticker):
    return parse_salary_data(fetch_salary_data(ticker), ticker)


# 並列処理で各企業に対してスクレイピングを実行
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    salary_data_list = list(executor.map(fetch_and_parse_salary_data, tickers))

# CSVファイルに出力
csv_filename = fr'C:\Users\100ca\Documents\PyCode\IR\data\IR.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['ticker', 'company_name', 'average_salary', 'average_age'])
    writer.writeheader()
    writer.writerows(salary_data_list)

print(f"Data saved to {csv_filename}")
salary_data_list

Data saved to C:\Users\100ca\Documents\PyCode\IR\data\IR.csv


[{'ticker': 1301,
  'company_name': '極洋',
  'average_salary': 725,
  'average_age': '40.8'},
 {'ticker': 1332,
  'company_name': 'ニッスイ',
  'average_salary': 798,
  'average_age': '43.01'},
 {'ticker': 1333,
  'company_name': 'マルハニチロ',
  'average_salary': 742,
  'average_age': '42.1'},
 {'ticker': 1375,
  'company_name': '雪国まいたけ',
  'average_salary': 399,
  'average_age': '41'},
 {'ticker': 1376,
  'company_name': 'カネコ種苗',
  'average_salary': 590,
  'average_age': '41.6'},
 {'ticker': 1377,
  'company_name': 'サカタのタネ',
  'average_salary': 683,
  'average_age': '38.7'},
 {'ticker': 1379,
  'company_name': 'ホクト',
  'average_salary': 512,
  'average_age': '38.9'},
 {'ticker': 1380,
  'company_name': '秋川牧園',
  'average_salary': 499,
  'average_age': '42.6'},
 {'ticker': 1381,
  'company_name': 'アクシーズ',
  'average_salary': 452,
  'average_age': '43.1'},
 {'ticker': 1382,
  'company_name': 'ホーブ',
  'average_salary': 421,
  'average_age': '41.5'},
 {'ticker': 1383,
  'company_name': 'ベルグアース',
 